# Generative StyleGAN 2 ADA

First thing, mount the Google drive allowing for this pytorch code to read/write to the desired Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# StyleGAN 2 ADA Installation

Running the next cell installs the StyleGAN 2 ADA repository. This can be installed now for later if the Google Drive isn't ready for mounting. 

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 4.2 MB/s 
     |████████████████████████████████| 776.8 MB 4.0 kB/s 
     |████████████████████████████████| 12.8 MB 58.4 MB/s 
     |████████████████████████████████| 17.3 MB 391 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages t

Running the next cell will update the previously installed StyleGAN 2 repository 

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Already up to date.
Saved working directory and index state WIP on main: 59e05bb added opensimplex


# Dataset

The next few cells will install the zipped dataset into the correct folder within StyleGAN 2. First upload the zipped dataset folder into /content/drive. Once complete run the next cell which will unzip the folder.

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/garments-dataset.zip') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/garments-dataset')

To verify all the images within the unzipped dataset, run the next cell. It will check the format for every single image within the dataset. Providing promts one issues, 4 tensors = 4 channels which is not accepted. The images must be RGB and follow the same pixel size as descripted within the downloaded pretrained model, ranging from 512 x 512 and 1024 x 1024. 

In [ ]:
# check size and colour format of images
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/garments-dataset/garments-dataset'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

If there are no issues...

Run the next cell, which will create the image destination folder within the StlyeGAN 2 Google Drive, which will use in a moment.  

In [ ]:
os.mkdir('/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/garments_square')

Now that the dataset folder has been created within the StyleGAN 2. It allows for the next cell to transfer and process the images with the pytorch dataset tool from the zipped folder into the StyleGAN 2 dataset folder. 

In [ ]:
!python /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/garments-dataset/garments-dataset --dest /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/garments_square

After selecting the desired pre-trained model from the resources, upload the PKL file into google drive. The next cell will transfer the PKL file into the correct StyleGAN 2 folder. /pretrained

In [ ]:
!mv '/content/drive/MyDrive/pretrained_model/network-snapshot-026392.pkl' '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained'

# Training the model

Now the pretrained PKL file and the images are in the correct folders within the StyleGAN 2. Training can take place using the uploaded PKL file as a base and then building ontop of it. 

Change the paths to match for specific applications. When start new training start from aug_strength 0 and train count 0. Running the next cell prepares the folders. 

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/garments_square/00000'
# resume_from = './pretrained/wikiart.pkl'
resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00006-00000-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000248.pkl'

aug_strength = 1.160
train_count = 248

mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 1

Running the next line start the training with the parameters set above. 


In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

The training session may cut, note down the aug_strength and train count. Adject the paramerts with the PKL file from the /results within StyleGAN 2 ADA.



# Testing

The next cell will generate images from the PKL file that's directed. Creating a single image for the PKL file. 

In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.8 --seeds=10 --network=''

generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
Traceback (most recent call last):
  File "generate.py", line 25, in <module>
    from opensimplex import OpenSimplex
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/__init__.py", line 4, in <module>
    from .api import *
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/api.py", line 2, in <module>
    from .internals import _init, _noise2, _noise3, _noise4, _noise2a, _noise3a, _noise4a
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/internals.py", line 9, in <module>
    from numba import njit, prange
  File "/usr/local/lib/python3.8/dist-packages/numba/__init__.py", line 42, in <module>
    from numba.np.ufunc import (vectorize, guvectorize, threading_layer,
  File "/usr/local/lib/python3.8/dist-packages/numba/np/ufunc/__init__.py", line 3, in <module>
    from numba.np.ufunc.decorators import Vectorize, GUVectorize, vectorize, guvectorize
  File "/

Custom pixel sized image outputs. 

In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.7 --size=1820-1024 --scale-type=symm --seeds=0-499 --network='/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00007-00000-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000308.pkl'

Realist images

In [ ]:
!python generate.py --process="truncation" --outdir=/content/out/trunc-trav-3/ --start=-0.8 --stop=2.8 --increment=0.02 --seeds=470 --network='./pretrained/network-snapshot-027750.pkl'

Linear Interpolation

In [ ]:
!python generate.py --outdir=/content/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=463,470 --network='./pretrained/network-snapshot-027750.pkl'

Slerp Interpolation

In [ ]:
!python generate.py --outdir=out/slerp-z/ --space="z" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network='./pretrained/network-snapshot-027750.pkl' --frames=24

Noise Loop

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network='./pretrained/network-snapshot-027750.pkl'

Circular Loop

In [ ]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network='./pretrained/network-snapshot-027750.pkl'